## Aggregate rasters
This is the workflow for aggregating predictor rasters to the coarser resolutions used for analysis (96.5 km, 193 km, and 385.9 km). Note- many of these aggregations use up a lot of memory that R doesn't efficiently dump. You may need to restart R in between aggregating some rasters to free up memory.

In [1]:
library(raster)
library(rnaturalearth)
library(here)
library(tictoc)
library(tidyverse)
library(furrr)
library(climateStability)

out_path <- here("data", "climate_agg")
res_list <- list(high = 96.5, medium = 193, low = 385.9)

Loading required package: sp

here() starts at /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks raster::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ dplyr::select()  masks raster::select()

Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:raster’:

    values




Create template rasters to resample predictors and filter genetic data with. 

Template resolutions:  
* 96.5 km (1 degree at 30 deg latitude) {**template_high**}
* 193 km (2 degrees) {**template_medium**}
* 385.9 km (4 degrees) {**template_low**}  


*Note that I am not aggregating predictor rasters to the low resolution. After exploring the genetic data, we determined that a low resolution is not compatible with our analyses. I'm leaving the template here because we used it to filter the genetic data*

In [2]:
crs <- "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs"

# read in a world shape file. Might make this able to use countries as well
world_shp <- rnaturalearth::ne_coastline(scale = "small")
  
# convert the coordinate system to behrmann
world_shp <- sp::spTransform(world_shp, crs)
  
# turn it into a raster
world_raster <- raster::raster(world_shp)


# transform world raster to high resolution
template_high <- world_raster
res(template_high) <- 96500


# transform world raster to medium resolution
template_medium <- world_raster
res(template_medium) <- 193000

# transform world raster to low resolution
template_low <- world_raster
res(template_low) <- 385900

# write to data/templates/ folder
if (!file.exists(here("data", "templates", "template_high.tif"))){
    writeRaster(template_high, filename = here("data", "templates", "template_high.tif"))
}

if (!file.exists(here("data", "templates", "template_medium.tif"))){
    writeRaster(template_medium, filename = here("data", "templates", "template_medium.tif"))
}

if (!file.exists(here("data", "templates", "template_low.tif"))){
    writeRaster(template_medium, filename = here("data", "templates", "template_low.tif"))
}

This function follows these steps:  
* reproject raster to Behrmann equal-area projection
* resample with bilinear interpolation to the coarse resolution

This process is slow, but results in rasters with the precise desired resolution in km. Alternatives result in imprecise resolutions, where for example a 96.5 km resolution specification, the resolution is actually 96.48628 km or 95 km. 

In [3]:
# function to resample rasters to any equal area grid of the three resolutions we're considering.


# crs: define the crs you want to project to. It must be an equal area projection. The default crs is behrmann equal area. 
# x: a raster or raster stack
# km: km resolution you want (area will be km x km)
# is_categorical: if is_categorical is true, using "ngb" interpolation
resample_equal_area <- function(x, 
                                crs = "+proj=cea +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +datum=WGS84 +ellps=WGS84 +units=m +no_defs", 
                                km,
                                is_categorical = FALSE) {
  
  # project raster to new coordinate system (this has no values- we put values back in later)
  x_ext <- raster::projectExtent(x, crs)
  
  # make the resolution square since we're interested in an equal area projection
  raster::res(x_ext) <- raster::xres(x_ext)
  
  # add original values to our projection. We can't take shortcuts like doing this instead of doing projectExtent first because it removes cells at the extreme latitudes.
  x_repro <- raster::projectRaster(x, x_ext)
  
  if (km == 96.5) {
      end_raster = template_high
  } else if (km == 193) {
      end_raster = template_medium
  } else if (km == 385.9) {
      end_raster = template_low
  } else stop("Incorrect resolution specification. Must be either 96.5 or 193")
  
  # resample our raster of interest to the beginning raster  
  if (is_categorical) {
      m <- "ngb"
  } else m <- "bilinear"
  
  
  x_resampled <- raster::resample(x_repro, end_raster, method = m) 

  
  return(x_resampled)
}


In [4]:
# function to write aggregated rasters to file, using a specified prefix for the file and layer names
write_raster <- function (x, prefix) {
    names(x) <- paste0(prefix, "_", names(x))
    out_file <- file.path(out_path, paste0(names(x), ".tif"))
    writeRaster(x, filename = out_file, overwrite = TRUE)
}

## Current climate
From [paleoclim.org](http://www.paleoclim.org/), based on [CHELSA](http://chelsa-climate.org/bioclim/). 2.5 arcmin resolution.

In [75]:
current_path <- dir(here("data", "climate_raw", "current_bioclim"), pattern = "*.tif$", full.names = TRUE)
current_rast_list <- purrr::map(current_path, raster)

#### 96.5 km (high) resolution

In [76]:
tic()
current_agg_high <- purrr::map(current_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(current_agg_high, ~write_raster(.x, prefix = "current_high")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto,

894.467 sec elapsed


#### 193 km (Medium) Resolution

In [95]:
tic()
current_agg_medium <- purrr::map(current_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(current_agg_medium, ~write_raster(.x, prefix = "current_medium")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto,

840.896 sec elapsed


## Late Holocene climate
(4.2-0.3 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [4]:
lh_path <- dir(here("data", "climate_raw", "late-holocene_bioclim"), pattern = "*.tif$", full.names = TRUE)
lh_rast_list <- purrr::map(lh_path, raster)

#### 96.5 km (high) resolution

In [7]:
tic()
lh_agg_high <- purrr::map(lh_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(lh_agg_high, ~write_raster(.x, prefix = "lh_high")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”


95.612 sec elapsed


#### 193 km (medium) resolution

In [8]:
tic()
lh_agg_medium <- purrr::map(lh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(lh_agg_medium, ~write_raster(.x, prefix = "lh_medium")))

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“51 projected point(s) not finite”


94.817 sec elapsed


## Mid Holocene climate

(8.326-4.2 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [9]:
mh_path <- dir(here("data", "climate_raw", "mid-holocene_bioclim"), pattern = "*.tif$", full.names = TRUE)
mh_rast_list <- purrr::map(mh_path, raster)

#### 96.5 km (high) resolution

In [10]:
tic()
mh_agg_high <- purrr::map(mh_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(mh_agg_high, ~write_raster(.x, prefix = "mh_high")))

93.949 sec elapsed


#### 193 km (medium) resolution

In [11]:
tic()
mh_agg_medium <- purrr::map(mh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(mh_agg_medium, ~write_raster(.x, prefix = "mh_medium")))

93.293 sec elapsed


## Early Holocene climate

(11.7-8.326 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [12]:
eh_path <- dir(here("data", "climate_raw", "early-holocene_bioclim"), pattern = "*.tif$", full.names = TRUE)
eh_rast_list <- purrr::map(eh_path, raster)

#### 96.5 km (high) resolution

In [13]:
tic()
eh_agg_high <- purrr::map(eh_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(eh_agg_high, ~write_raster(.x, prefix = "eh_high")))

93.762 sec elapsed


#### 193 km (medium) resolution

In [14]:
tic()
eh_agg_medium <- purrr::map(eh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(eh_agg_medium, ~write_raster(.x, prefix = "eh_medium")))

93.244 sec elapsed


## Younger Dryas Stadial climate

(12.9-11.7 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [15]:
yds_path <- dir(here("data", "climate_raw", "younger-dryas_bioclim"), pattern = "*.tif$", full.names = TRUE)
yds_rast_list <- purrr::map(yds_path, raster)

#### 96.5 km (high) resolution

In [16]:
tic()
yds_agg_high <- purrr::map(yds_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(yds_agg_high, ~write_raster(.x, prefix = "yds_high")))

93.859 sec elapsed


#### 193 km (medium) resolution

In [17]:
tic()
yds_agg_medium <- purrr::map(mh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(yds_agg_medium, ~write_raster(.x, prefix = "yds_medium")))

93.253 sec elapsed


## Bølling-Allerød climate

(14.7-12.9 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [18]:
ba_path <- dir(here("data", "climate_raw", "bolling-allerod_bioclim"), pattern = "*.tif$", full.names = TRUE)
ba_rast_list <- purrr::map(ba_path, raster)

#### 96.5 km (high) resolution

In [19]:
tic()
ba_agg_high <- purrr::map(ba_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(ba_agg_high, ~write_raster(.x, prefix = "ba_high")))

93.505 sec elapsed


#### 193 km (medium) resolution

In [20]:
tic()
ba_agg_medium <- purrr::map(ba_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(ba_agg_medium, ~write_raster(.x, prefix = "ba_medium")))

93.794 sec elapsed


## Heinrich Stadial 1 climate

(17.0-14.7 ka)

From [paleoclim.org](paleoclim.org). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [21]:
hs_path <- dir(here("data", "climate_raw", "heinrich-stadial_bioclim"), pattern = "*.tif$", full.names = TRUE)
hs_rast_list <- purrr::map(hs_path, raster)

#### 96.5 km (high) resolution

In [22]:
tic()
hs_agg_high <- purrr::map(hs_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(hs_agg_high, ~write_raster(.x, prefix = "hs_high")))

94.057 sec elapsed


#### 193 km (medium) resolution

In [23]:
tic()
hs_agg_medium <- purrr::map(hs_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(hs_agg_medium, ~write_raster(.x, prefix = "hs_medium")))

93.529 sec elapsed


## LGM climate
From [paleoclim.org](paleoclim.org), based on [CHELSA](http://chelsa-climate.org/last-glacial-maximum-climate/). 2.5 arcmin resolution. Only using BIO1 (average annual temp) and BIO12 (average annual precipitation) for stability estimation.

In [28]:
lgm_path <- dir(here("data", "climate_raw", "LGM_bioclim"), pattern = "*.tif$", full.names = TRUE)
lgm_rast_list <- purrr::map(lgm_path, raster)

#### 96.5 km (high) resolution

In [29]:
tic()
lgm_agg_high <- purrr::map(lgm_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(lgm_agg_high, ~write_raster(.x, prefix = "lgm_high")))

93.601 sec elapsed


#### 193 km (medium) resolution

In [30]:
tic()
lgm_agg_medium <- purrr::map(lgm_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(lgm_agg_medium, ~write_raster(.x, prefix = "lgm_medium")))

92.793 sec elapsed


## Soil
Soil classifications from the [Harmonized world soil database](http://www.fao.org/soils-portal/soil-survey/soil-maps-and-databases/harmonized-world-soil-database-v12/en/). 30 arc-second resolution.  
 
Citation: Fischer, G., F. Nachtergaele, S. Prieler, H.T. van Velthuizen, L. Verelst, D. Wiberg, 2008. Global Agro-ecological Zones Assessment for Agriculture (GAEZ 2008). IIASA, Laxenburg, Austria and FAO, Rome, Italy.

In [5]:
soil_path <- dir(here("data", "climate_raw", "HWSD_soil_classification"), pattern = "*.bil$", full.names = TRUE)
soil_rast_list <- raster(soil_path)
crs(soil_rast_list) <- "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"

#### 96.5 km (high) resolution

In [63]:
tic()
soil_agg_high <- resample_equal_area(soil_rast_list, km = res_list$high, is_categorical = TRUE)
toc()
# write rasters to file
write_raster(soil_agg_high, prefix = "soil_high")

1431.368 sec elapsed


#### 193 km (medium) resolution

In [6]:
tic()
soil_agg_medium <- resample_equal_area(soil_rast_list, km = res_list$medium, is_categorical = TRUE)
toc()
# write rasters to file
write_raster(soil_agg_medium, prefix = "soil_medium")

2490.804 sec elapsed


## Terrain
Elevation raster from [worldclim.org](worldclim.org) (based on SRTM elevation data). 2.5 arcmin resolution. I used the `raster::terrain()` function to calculate:  
* slope
* aspect
* Topographic Position Index (TPI)
* Terrain Ruggedness Index (TRI)
* roughness

The terrain calculation script is located here: `R/calc_terrain.R`

In [83]:
terrain_path <- dir(here("data", "climate_raw", "terrain"), pattern = "*.tif$", full.names = TRUE)
terrain_rast_list <- purrr::map(terrain_path, raster)

#### 96.5 km (high) resolution

In [84]:
tic()
terrain_agg_high <- purrr::map(terrain_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(terrain_agg_high, ~write_raster(.x, prefix = "terrain_high")))

291.247 sec elapsed


#### 193 km (medium) resolution

In [85]:
tic()
terrain_agg_medium <- purrr::map(terrain_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(terrain_agg_medium, ~write_raster(.x, prefix = "terrain_medium")))

280.117 sec elapsed


## Habitat heterogeneity
Global habitat heterogeneity measures from [earthenv.org](earthenv.org). 2.5 arcmin resolution. These are first and second-order texture indices based on the Enhanced Vegetation Index. The publication for these is [Tuanmu and Jetz 2015](http://onlinelibrary.wiley.com/doi/10.1111/geb.12365/abstract). 

In [88]:
ghh_path <- dir(here("data", "climate_raw", "ghh"), pattern = "*.tif$", full.names = TRUE)
ghh_rast_list <- purrr::map(ghh_path, raster)

#### 96.5 km (high) resolution

In [89]:
tic()
ghh_agg_high <- purrr::map(ghh_rast_list, ~resample_equal_area(.x, km = res_list$high))
toc()
# write rasters to file
invisible(purrr::map(ghh_agg_high, ~write_raster(.x, prefix = "ghh_high")))

608.204 sec elapsed


#### 193 km (medium) resolution

In [94]:
tic()
ghh_agg_medium <- purrr::map(ghh_rast_list, ~resample_equal_area(.x, km = res_list$medium))
toc()
# write rasters to file
invisible(purrr::map(ghh_agg_medium, ~write_raster(.x, prefix = "ghh_medium")))

571.337 sec elapsed


## Dynamic Habitat Indices
Dynamic Habitat Indices from [the Sylvis Lab](http://silvis.forest.wisc.edu/data/dhis/). 30 arcsecond resolution. There are three vegetation measures: cumulative productivity (band 1), minimum productivity (band 2), and inter-annual variation of productivity (band 3). They are all based on MODIS-derived Enhanced Vegetation Indicies. The publication for this is [Hobi et al. 2017](https://doi.org/10.1016/j.rse.2017.04.018). Since the raw files are kept as a raster stack, I'm using a for loop instead of `purrr::map`.

In [50]:
dhi_path <- dir(here("data", "climate_raw", "dhi"), pattern = "*.tif$", full.names = TRUE)
dhi_rast_list <- stack(dhi_path)

# assign reasonable names to each band
names(dhi_rast_list) <- c("cum", "min", "var")

#### 96.5 km (high) resolution

In [51]:
tic()
for(i in 1:3) {
    dhi_agg_high <- resample_equal_area(dhi_rast_list[[i]], km = res_list$high)
    write_raster(dhi_agg_high, prefix = "dhi_high")
}
toc()

3669.726 sec elapsed


#### 193 km (medium) resolution

In [52]:
tic()
for(i in 1:3) {
    dhi_agg_medium <- resample_equal_area(dhi_rast_list[[i]], km = res_list$medium)
    write_raster(dhi_agg_medium, prefix = "dhi_medium")
}
toc()

3718.069 sec elapsed


## Vascular plant phylogenetic diversity
Predicted vascular plant species richness across the globe. This is the combined model from [Kreft and Jetz 2007]( https://doi.org/10.1073/pnas.0608361104). 110km resolution. Since the high resolution is only slightly higher resolution, I think it's okay to use bilinear interpolation to downscale. The covariates necessary to perform more complex machine learning-based downscaling have to be aggregated to 96.5 km, so the accuracy and precision gain would likely be minimal.

In [45]:
plant_path <- dir(here("data", "climate_raw", "plants"), pattern = "*.tif$", full.names = TRUE)
plant_rast_list <- raster(plant_path)

#### 96.5 km (high) resolution

In [48]:
tic()
plant_agg_high <- resample_equal_area(plant_rast_list, km = res_list$high)
toc()
# write rasters to file
write_raster(plant_agg_high, prefix = "plant_high")

0.751 sec elapsed


#### 193 km (medium) resolution

In [49]:
tic()
plant_agg_medium <- resample_equal_area(plant_rast_list, km = res_list$medium)
toc()
# write rasters to file
write_raster(plant_agg_medium, prefix = "plant_medium")

0.302 sec elapsed


## Human modification
0-1 scaled measure of human modification of terrestrial lands across the globe. 30 arc-second resolution.
Citation: Kennedy CM, Oakleaf JR, Theobald DM, Baruch‐Mordo S, Kiesecker J. Managing the middle: A shift in conservation priorities based on the global human modification gradient. Glob Change Biol. 2019;25:811–826. https://doi.org/10.1111/gcb.14549

In [6]:
human_path <- dir(here("data", "climate_raw", "human_mod"), pattern = "*.tif$", full.names = TRUE)
human_rast_list <- raster(human_path)

#### 96.5 km (high) resolution

In [6]:
tic()
human_agg_high <- resample_equal_area(human_rast_list, km = res_list$high)
toc()
# write rasters to file
write_raster(human_agg_high, prefix = "human_high")

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”


954.678 sec elapsed


#### 193 km (medium) resolution

In [7]:
tic()
human_agg_medium <- resample_equal_area(human_rast_list, km = res_list$medium)
toc()
# write rasters to file
write_raster(human_agg_medium, prefix = "human_medium")

Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”
Warning message in rgdal::rawTransform(projfrom, projto, nrow(xy), xy[, 1], xy[, :
“230 projected point(s) not finite”


961.798 sec elapsed


## Estimate climate stability since the LGM
I am using the [climateStability R package](https://cran.r-project.org/web/packages/climateStability/index.html) to estimate climate stability since the LGM. I am summarizing the aggregated BIO1 (average annual temperature) and BIO12 (average annual precipitation) bioclim variables for the 8 time slices mentioned previously.

In [45]:
### Create subdirectories for temperature and stability rasters I'm going to estimate stability with

# temperature
if (!dir.exists(here("data", "climate_agg", "stability_temp_high"))) {
    dir.create(here("data", "climate_agg", "stability_temp_high"))
}

if (!dir.exists(here("data", "climate_agg", "stability_temp_medium"))) {
    dir.create(here("data", "climate_agg", "stability_temp_medium"))
}


# precipitation
if (!dir.exists(here("data", "climate_agg", "stability_precip_high"))) {
    dir.create(here("data", "climate_agg", "stability_precip_high"))
}

if (!dir.exists(here("data", "climate_agg", "stability_precip_medium"))) {
    dir.create(here("data", "climate_agg", "stability_precip_medium"))
}

# get filenames to copy
temp_high <- list.files(here("data", "climate_agg"), pattern = "_high_bio_1.tif", full.names = TRUE)
precip_high <- list.files(here("data", "climate_agg"), pattern = "_high_bio_12.tif", full.names = TRUE)

temp_medium <- list.files(here("data", "climate_agg"), pattern = "_medium_bio_1.tif", full.names = TRUE)
precip_medium <- list.files(here("data", "climate_agg"), pattern = "_medium_bio_12.tif", full.names = TRUE)

# copy climate files to subfolders
file.copy(temp_high, here("data", "climate_agg", "stability_temp_high"))
file.copy(precip_high, here("data", "climate_agg", "stability_precip_high"))
file.copy(temp_medium, here("data", "climate_agg", "stability_temp_medium"))
file.copy(precip_medium, here("data", "climate_agg", "stability_precip_medium"))

[1] FALSE  TRUE

[1] FALSE  TRUE

[1] FALSE  TRUE

[1] FALSE  TRUE

In [46]:
# remove these files from climate_agg directory (except for current) so they don't get included as predictors
temp_high_noc <- temp_high[!str_detect(temp_high, "current")]
precip_high_noc <- precip_high[!str_detect(precip_high, "current")]
temp_medium_noc <- temp_medium[!str_detect(temp_medium, "current")]
precip_medium_noc <- precip_medium[!str_detect(precip_medium, "current")]

all_noc <- c(temp_high_noc, precip_high_noc, temp_medium_noc, precip_medium_noc)
file.remove(all_noc)

[1] TRUE TRUE TRUE TRUE

I have to rename the files because the `climateStability` package needs the files to be in order when they're read in. I'm prepending numbers to each file name so they are ordered correctly when they're read in.

In [55]:
rename_files <- function(file){
    file_base <- basename(file)
    file_dir <- dirname(file)
    new_name <- case_when(
        str_detect(file_base, "^current") ~ file.path(file_dir, paste0("1_", file_base)),
        str_detect(file_base, "^lh") ~ file.path(file_dir, paste0("2_", file_base)),
        str_detect(file_base, "^mh") ~ file.path(file_dir, paste0("3_", file_base)),
        str_detect(file_base, "^eh") ~ file.path(file_dir, paste0("4_", file_base)),
        str_detect(file_base, "^yds") ~ file.path(file_dir, paste0("5_", file_base)),
        str_detect(file_base, "^ba") ~ file.path(file_dir, paste0("6_", file_base)),
        str_detect(file_base, "^hs") ~ file.path(file_dir, paste0("7_", file_base)),
        str_detect(file_base, "^lgm") ~ file.path(file_dir, paste0("8_", file_base))
    )
    
    file.rename(file, new_name)
}

# stability_temp_high
purrr::map(list.files(here("data", "climate_agg", "stability_temp_high"), full.names = TRUE), rename_files)

# stability_precip_high
purrr::map(list.files(here("data", "climate_agg", "stability_precip_high"), full.names = TRUE), rename_files)

# stability_temp_medium
purrr::map(list.files(here("data", "climate_agg", "stability_temp_medium"), full.names = TRUE), rename_files)

# stability_precip_medium
purrr::map(list.files(here("data", "climate_agg", "stability_precip_medium"), full.names = TRUE), rename_files)


[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

Now I have to convert each raster to a `.asc` file for `climateStability` to use. I'm moving the `.tif` files to another folder just in case I find out later the conversion botched the files. They're small (~100-120 kB), so it's not a major cost to have duplicates.

In [61]:
convert_to_asc <- function(rast) {
    if(str_detect(rast, ".tif")) {
        s <- str_replace(rast, ".tif", ".asc")
        r <- raster(rast)
        writeRaster(r, s)
    } else message("Already an ascii file")    
}

# stability_temp_high
purrr::map(list.files(here("data", "climate_agg", "stability_temp_high"), full.names = TRUE), convert_to_asc)

# stability_precip_high
purrr::map(list.files(here("data", "climate_agg", "stability_precip_high"), full.names = TRUE), convert_to_asc)

# stability_temp_medium
purrr::map(list.files(here("data", "climate_agg", "stability_temp_medium"), full.names = TRUE), convert_to_asc)

# stability_precip_medium
purrr::map(list.files(here("data", "climate_agg", "stability_precip_medium"), full.names = TRUE), convert_to_asc)


[[1]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_temp_high/1_current_high_bio_1.asc 
names      : X1_current_high_bio_1 


[[2]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_temp_high/2_lh_high_bio_1.asc 
names      : X2_lh_high_bio_1 


[[3]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax)
crs   

[[1]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_precip_high/1_current_high_bio_12.asc 
names      : X1_current_high_bio_12 


[[2]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_precip_high/2_lh_high_bio_12.asc 
names      : X2_lh_high_bio_12 


[[3]]
class      : RasterLayer 
dimensions : 151, 360, 54360  (nrow, ncol, ncell)
resolution : 96500, 96500  (x, y)
extent     : -17367530, 17372470, -7274787, 7296713  (xmin, xmax, ymin, ymax

[[1]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_temp_medium/1_current_medium_bio_1.asc 
names      : X1_current_medium_bio_1 


[[2]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_temp_medium/2_lh_medium_bio_1.asc 
names      : X2_lh_medium_bio_1 


[[3]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, xmax, ymi

[[1]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_precip_medium/1_current_medium_bio_12.asc 
names      : X1_current_medium_bio_12 


[[2]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, xmax, ymin, ymax)
crs        : NA 
source     : /Users/labuser/Desktop/connorfrench/bigass-phylogeography/BigAss-phylogeography/data/climate_agg/stability_precip_medium/2_lh_medium_bio_12.asc 
names      : X2_lh_medium_bio_12 


[[3]]
class      : RasterLayer 
dimensions : 76, 180, 13680  (nrow, ncol, ncell)
resolution : 193000, 193000  (x, y)
extent     : -17367530, 17372470, -7371287, 7296713  (xmin, x

In [65]:
file_move <- function(from, to) {
    file_base <- basename(from)
    new_file <- file.path(to, file_base)
    file.rename(from, new_file)
}


if(!dir.exists(here("data", "climate_agg", "stability_temp_high_tifs"))) {
    dir.create(here("data", "climate_agg", "stability_temp_high_tifs"))
}

if(!dir.exists(here("data", "climate_agg", "stability_precip_high_tifs"))) {
    dir.create(here("data", "climate_agg", "stability_precip_high_tifs"))
}

if(!dir.exists(here("data", "climate_agg", "stability_temp_medium_tifs"))) {
    dir.create(here("data", "climate_agg", "stability_temp_medium_tifs"))
}
if(!dir.exists(here("data", "climate_agg", "stability_precip_medium_tifs"))) {
    dir.create(here("data", "climate_agg", "stability_precip_medium_tifs"))
}


# stability_temp_high
stability_temp_high_df <- expand_grid(
    list.files(here("data", "climate_agg", "stability_temp_high"), pattern = ".tif$", full.names = TRUE),
    here("data", "climate_agg", "stability_temp_high_tifs")
)
purrr::map2(stability_temp_high_df[,1], 
            stability_temp_high_df[,2], 
            file_move)

# stability_precip_high
stability_precip_high_df <- expand_grid(
    list.files(here("data", "climate_agg", "stability_precip_high"), pattern = ".tif$", full.names = TRUE),
    here("data", "climate_agg", "stability_precip_high_tifs")
)

purrr::map2(stability_precip_high_df[,1], 
            stability_precip_high_df[,2], 
            file_move)


# stability_temp_medium
stability_temp_medium_df <- expand_grid(
    list.files(here("data", "climate_agg", "stability_temp_medium"), pattern = ".tif$", full.names = TRUE),
    here("data", "climate_agg", "stability_temp_medium_tifs")
)

purrr::map2(stability_temp_medium_df[,1], 
            stability_temp_medium_df[,2], 
            file_move)

# stability_precip_medium
stability_precip_medium_df <- expand_grid(
    list.files(here("data", "climate_agg", "stability_precip_medium"), pattern = ".tif$", full.names = TRUE),
    here("data", "climate_agg", "stability_precip_medium_tifs")
)

purrr::map2(stability_precip_medium_df[,1], 
            stability_precip_medium_df[,2], 
            file_move)


$`list.files(...)`
[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

$`list.files(...)`
[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

$`list.files(...)`
[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

$`list.files(...)`
[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

Estimate stability for each variable-resolution pair and rescale from 0-1. I found a bug in the `deviationThroughTime` code, where the full paths of the variable directory aren't returned (didn't include `full.paths = TRUE` argument in `list.files()`). So, I'm slightly modifying the function here to work with my code. I submitted a bug report, so hopefully it gets fixed!

In [4]:
# originally deviationThroughTime from climateStability R package
deviation_through_time <- function (variableDirectory, timeSlicePeriod, fileExtension = "asc") 
{
    match.arg(fileExtension, c("grd", "asc", "sdat", "rst", "nc", 
        "tif", "envi", "bil", "img"))
    if (!dir.exists(variableDirectory)) {
        stop(variableDirectory, " is not a directory that exists.")
    }
    if (!is.numeric(timeSlicePeriod)) {
        stop(timeSlicePeriod, " is not numeric.")
    }
    # this is where I added "full.names = TRUE"
    rastList <- list.files(path = variableDirectory, pattern = paste0(".", 
        fileExtension, "$"), full.names = TRUE)
    if (length(timeSlicePeriod) != 1 && length(timeSlicePeriod) != 
        (length(rastList) - 1)) {
        stop("The specified timeSlicePeriod object is not of expected length (1 or one less than the number of .asc files in variableDirectory)")
    }
    varStack <- raster::stack(rastList)
    intervalDev <- varStack[[-1]]
    count <- 2
    while (count <= length(rastList)) {
        if (length(timeSlicePeriod) == 1) {
            intervalDev[[(count - 1)]] <- raster::calc(varStack[[(count - 
                1):count]], sd)/timeSlicePeriod[[1]]
        }
        else {
            intervalDev[[(count - 1)]] <- raster::calc(varStack[[(count - 
                1):count]], sd)/timeSlicePeriod[[(count - 1)]]
        }
        count <- count + 1
    }
    if (length(timeSlicePeriod) == 1) {
        deviation <- sum(intervalDev)/(timeSlicePeriod * (length(rastList) - 
            1))
    }
    else {
        deviation <- sum(intervalDev)/sum(timeSlicePeriod)
    }
    return(deviation)
}

In [5]:
### Creating temperature and precipitation stability layers
### Using bio1 (average temp) and bio12 (average precip)

# time slices are calculated as the difference between the midpoints of the two time periods the climate layers were calculated for (e.g. midpoint of LH = (4.2 ka - 0.3 ka) / 2 + 0.3 ka = 2.25, midpoint of MH = (8.326 ka - 4.2 ka) / 2 + 4.2 = 6.263. time_slice = 6.263 - 2.25 = 4.013)

# c = current
# t_lh = midpoint time for late holocene
# t_mh = mid-Holocene
# t_eh = early-Holocene
# t_yds = Younger Dryas Stadial
# t_ba = Bølling-Allerød
# t_hs = Heinrich Stadial 1
# t_lgm = Last Glacial Maximum
c <- (2013 - 1979) / 2
t_lh <- ((4.2 - 0.3) / 2 + 0.3) * 1000
t_mh <- ((8.326 - 4.2) / 2 + 4.2) * 1000
t_eh <- ((11.7 - 8.326) / 2 + 8.326) * 1000
t_yds <- ((12.9 - 11.7) / 2 + 11.7) * 1000
t_ba <- ((14.7 - 12.9) / 2 + 12.9) * 1000
t_hs <- ((17 - 14.7) / 2 + 14.7) * 1000
t_lgm <- 21000

time_slices <- c(t_lh - c, 
                 t_mh - t_lh, 
                 t_eh - t_mh, 
                 t_yds - t_eh, 
                 t_ba - t_yds, 
                 t_hs - t_ba,
                 t_lgm - t_hs)

precip_path_high <- here("data", "climate_agg", "stability_precip_high")
temp_path_high <- here("data", "climate_agg", "stability_temp_high")
precip_path_medium <- here("data", "climate_agg", "stability_precip_medium")
temp_path_medium <- here("data", "climate_agg", "stability_temp_medium")

### Precip stability for high resolution ###
precip_deviation_high <-
  deviation_through_time(variableDirectory = precip_path_high, timeSlicePeriod = time_slices)
precip_stability_high <- (1 / precip_deviation_high)
precip_stability_scaled_high <- climateStability::rescale0to1(precip_stability_high)

# write precipitation stability to file
writeRaster(
  precip_stability_scaled_high,
  filename = file.path(here("data", "climate_agg"), "stability_precip_high.tif"),
  format = "GTiff",
  overwrite = TRUE
)


###########################################

### Temp stability for high resolution ###
temp_deviation_high <-
  deviation_through_time(variableDirectory = temp_path_high, timeSlicePeriod = time_slices)
temp_stability_high <- (1 / temp_deviation_high)
temp_stability_scaled_high <- climateStability::rescale0to1(temp_stability_high)

# write precipitation stability to file
writeRaster(
  temp_stability_scaled_high,
  filename = file.path(here("data", "climate_agg"), "stability_temp_high.tif"),
  format = "GTiff",
  overwrite = TRUE
)

###########################################

### Precip stability for medium resolution ###
precip_deviation_medium <-
  deviation_through_time(variableDirectory = precip_path_medium, timeSlicePeriod = time_slices)
precip_stability_medium <- (1 / precip_deviation_medium)
precip_stability_scaled_medium <- climateStability::rescale0to1(precip_stability_medium)

# write precipitation stability to file
writeRaster(
  precip_stability_scaled_medium,
  filename = file.path(here("data", "climate_agg"), "stability_precip_medium.tif"),
  format = "GTiff",
  overwrite = TRUE
)

###########################################

### Temp stability for medium resolution ###
temp_deviation_medium <-
  deviation_through_time(variableDirectory = temp_path_medium, timeSlicePeriod = time_slices)
temp_stability_medium <- (1 / temp_deviation_medium)
temp_stability_scaled_medium <- climateStability::rescale0to1(temp_stability_medium)

# write precipitation stability to file
writeRaster(
  temp_stability_scaled_medium,
  filename = file.path(here("data", "climate_agg"), "stability_temp_medium.tif"),
  format = "GTiff",
  overwrite = TRUE
)

###########################################